In [1]:
# Import libraries and load data
import pandas as pd
import re

# Load the dataset
df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Task\Matching_WebScrape\4_5.csv')

# Filter for specific states and Exit type addresses
#df = df[df['OCR_state'].isin(['CA', 'UT', 'NV', 'AZ'])]
#df = df[df['OCR_Address_Type'] == 'empty']
#filter flag_reason Flag_Reason No matches found
#df = df[df['Flag_Reason'] == 'No matches found']
#print(f"Data loaded: {len(df)} exit records in CA, UT, NV, AZ")
df[['OCR_address_standardized_OFF_parenthesis', 'OCR_Exit_Number', 'Flagged', 'Flag_Reason','OCR_Address_Type']]

C:\Users\clint\AppData\Local\Temp\ipykernel_25368\3414597551.py:6: DtypeWarning: Columns (19,36,53,57,72,74,77,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Task\Matching_WebScrape\4_5.csv')


,OCR_address_standardized_OFF_parenthesis,OCR_Exit_Number,Flagged,Flag_Reason,OCR_Address_Type
0,I-90,NaN,False,Available in RVer and Trucker,empty
1,I-90,NaN,False,Available in RVer and Trucker,empty
2,I-90,NaN,False,Available in RVer and Trucker,empty
3,I-90,NaN,False,Available in RVer and Trucker,empty
4,I-90,NaN,False,Available in RVer and Trucker,empty
...,...,...,...,...,...
67016,AK 1,NaN,True,No matches found,empty
67017,AK 1,NaN,True,No matches found,empty
67018,AK Hwy 2,NaN,True,No matches found,empty
67019,AK Hwy 2,NaN,True,No matches found,empty


In [2]:
# Updated version of the extract_road_info function to handle diverse road types
def extract_road_info_fixed(address):
    """
    Extract main road and secondary road information from address text.
    Places main road in OCR_Main_Road and secondary road in OCR_Secondary_Road.
    If only one road exists, puts it in OCR_Main_Road and leaves OCR_Secondary_Road blank.
    """
    if pd.isna(address) or address == '':
        return '', ''
    
    # Clean the address - remove extra spaces and convert to string
    address = str(address).strip()
    
    # Initialize output variables
    main_road = ''
    secondary_road = ''
    
    # Handle hyphenated roads first (like CA 29-175)
    hyphen_pattern = r'\b((?:CA|US|I-|Hwy|SR)\s*\d+[-]\d+)\b'
    hyphen_match = re.search(hyphen_pattern, address, re.IGNORECASE)
    if hyphen_match and '&' not in address:
        main_road = hyphen_match.group(1).strip()
        return main_road, secondary_road
    
    # Pattern for roads with & separator (two roads)
    if '&' in address:
        parts = address.split('&', 1)  # Split only on first &
        first_part = parts[0].strip()
        second_part = parts[1].strip()
        
        # Extract main road from first part
        main_road = extract_single_road(first_part)
        
        # Extract secondary road from second part
        secondary_road = extract_single_road(second_part)
        
    else:
        # Single road - extract main road only
        main_road = extract_single_road(address)
    
    return main_road, secondary_road

def extract_single_road(road_text):
    """
    Helper function to extract a single road from text
    """
    if not road_text or road_text.strip() == '':
        return ''
    
    road_text = road_text.strip()
    
    # Priority patterns - highways and numbered routes first
    highway_patterns = [
        r'\b(Hwy\s+\d+(?:\s+[NSEW])?)',          # Hwy 67, Hwy 65 W
        r'\b(US\s+\d+)',                         # US 101
        r'\b(CA\s+\d+(?:-\d+)?)',               # CA 152, CA 29-175
        r'\b(I-\d+(?:-\d+)?)',                  # I-15, I-15-84
        r'\b(SR\s+\d+)',                        # SR 99
        r'\b(UT\s+\d+)',                        # UT routes
        r'\b(NV\s+\d+)',                        # NV routes
        r'\b(AZ\s+\d+)',                        # AZ routes
    ]
    
    # Check for highway patterns first
    for pattern in highway_patterns:
        match = re.search(pattern, road_text, re.IGNORECASE)
        if match:
            return match.group(1).strip()
    
    # Pattern for abbreviated roads with numbers (like "Rd 4", "St 5") - check BEFORE numbered streets
    abbrev_road_pattern = r'\b((?:Rd|St|Ave|Blvd|Dr|Ln|Way|Pkwy)\s+\w+)\b'
    abbrev_match = re.search(abbrev_road_pattern, road_text, re.IGNORECASE)
    if abbrev_match:
        return abbrev_match.group(1).strip()
    
    # Pattern for numbered streets/avenues (like "4th St", "12th Ave")
    numbered_street_pattern = r'\b(\d+(?:st|nd|rd|th)\s+(?:St|Street|Ave|Avenue|Rd|Road|Blvd|Boulevard))\b'
    numbered_match = re.search(numbered_street_pattern, road_text, re.IGNORECASE)
    if numbered_match:
        return numbered_match.group(1).strip()
    
    # Pattern for named streets/roads (like "Main St", "Riverford Rd")
    street_pattern = r'\b([A-Za-z]+(?:\s+[A-Za-z]+)*)\s+(St|Street|Ave|Avenue|Rd|Road|Blvd|Boulevard|Dr|Drive|Ln|Lane|Way|Pkwy|Parkway)\b'
    street_match = re.search(street_pattern, road_text, re.IGNORECASE)
    if street_match:
        street_name = street_match.group(1).strip()
        suffix = street_match.group(2).strip()
        return f"{street_name} {suffix}"
    
    # If no specific pattern found, try to extract meaningful content
    # Clean up common non-essential words
    exclude_words = {'exit', 'jct', 'junction', 'at', 'near', 'and', 'the', 'on', 'off', 'to', 'from'}
    
    # Split into words and filter
    words = re.findall(r'\b\w+\b', road_text)
    filtered_words = [word for word in words if word.lower() not in exclude_words and word.strip()]
    
    if filtered_words:
        # Return the full filtered content for short cases (like single names)
        if len(filtered_words) <= 2:
            return ' '.join(filtered_words)
        # For longer cases, take the first 2 words
        else:
            return ' '.join(filtered_words[:2])
    
    return ''

# Test the function with sample address patterns
test_cases = [
    "12th Ave",
    "Alameda & Eubanks", 
    "Hwy 67 & Riverford Rd",
    "Hwy 65 W & Henderson",
    "US 101 & 4th St",
    "CA 152 & Rd 4",
    "CA 29-175"
]

print("Testing road extraction function:")
print("=" * 50)
for test_case in test_cases:
    main, secondary = extract_road_info_fixed(test_case)
    print(f"Input: '{test_case}'")
    print(f"  OCR_Main_Road: '{main}'")
    print(f"  OCR_Secondary_Road: '{secondary}'")
    print("-" * 30)

Testing road extraction function:
Input: '12th Ave'
  OCR_Main_Road: '12th Ave'
  OCR_Secondary_Road: ''
------------------------------
Input: 'Alameda & Eubanks'
  OCR_Main_Road: 'Alameda'
  OCR_Secondary_Road: 'Eubanks'
------------------------------
Input: 'Hwy 67 & Riverford Rd'
  OCR_Main_Road: 'Hwy 67'
  OCR_Secondary_Road: 'Riverford Rd'
------------------------------
Input: 'Hwy 65 W & Henderson'
  OCR_Main_Road: 'Hwy 65 W'
  OCR_Secondary_Road: 'Henderson'
------------------------------
Input: 'US 101 & 4th St'
  OCR_Main_Road: 'US 101'
  OCR_Secondary_Road: '4th St'
------------------------------
Input: 'CA 152 & Rd 4'
  OCR_Main_Road: 'CA 152'
  OCR_Secondary_Road: 'Rd 4'
------------------------------
Input: 'CA 29-175'
  OCR_Main_Road: 'CA 29-175'
  OCR_Secondary_Road: ''
------------------------------


In [3]:
# Apply the function to extract road information
print("Extracting road information from dataset...")
road_info = df['OCR_address_standardized_OFF_parenthesis'].apply(extract_road_info_fixed)

# Update the columns with the extraction results
df['OCR_Main_Road'] = [info[0] for info in road_info]
df['OCR_Secondary_Road'] = [info[1] for info in road_info]

# Display results
print(f"Extraction complete!")
print(f"Records with main road extracted: {(df['OCR_Main_Road'] != '').sum()}")
print(f"Records with secondary road extracted: {(df['OCR_Secondary_Road'] != '').sum()}")

# Show some sample results
sample_records = df[df['OCR_Main_Road'] != ''].head(10)
if len(sample_records) > 0:
    print(f"\nSample extraction results:")
    print("=" * 80)
    for idx, row in sample_records.iterrows():
        print(f"Input: '{row['OCR_address_standardized_OFF_parenthesis']}'")
        print(f"  OCR_Main_Road: '{row['OCR_Main_Road']}'")
        print(f"  OCR_Secondary_Road: '{row['OCR_Secondary_Road']}'")
        print("-" * 60)

Extracting road information from dataset...
Extraction complete!
Records with main road extracted: 66951
Records with secondary road extracted: 12109

Sample extraction results:
Input: 'I-90'
  OCR_Main_Road: 'I-90'
  OCR_Secondary_Road: ''
------------------------------------------------------------
Input: 'I-90'
  OCR_Main_Road: 'I-90'
  OCR_Secondary_Road: ''
------------------------------------------------------------
Input: 'I-90'
  OCR_Main_Road: 'I-90'
  OCR_Secondary_Road: ''
------------------------------------------------------------
Input: 'I-90'
  OCR_Main_Road: 'I-90'
  OCR_Secondary_Road: ''
------------------------------------------------------------
Input: 'I-90'
  OCR_Main_Road: 'I-90'
  OCR_Secondary_Road: ''
------------------------------------------------------------
Input: 'I-90'
  OCR_Main_Road: 'I-90'
  OCR_Secondary_Road: ''
------------------------------------------------------------
Input: 'I-90'
  OCR_Main_Road: 'I-90'
  OCR_Secondary_Road: ''
---------------

In [4]:
# Save the updated dataframe with road information extraction
output_file = r'C:\Users\clint\Desktop\Geocoding_Task\Matching_WebScrape\4_6.csv'
df.to_csv(output_file, index=False)
print(f"Data with road extraction saved to: {output_file}")


Data with road extraction saved to: C:\Users\clint\Desktop\Geocoding_Task\Matching_WebScrape\4_6.csv
